<a href="https://colab.research.google.com/github/OdysseusPolymetis/chr2023/blob/main/3_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**WORD VECTORS**</center>

In [8]:
import os
import gensim
from gensim.models import Word2Vec
import glob
import nltk

from lxml import etree as ET
import lxml.html
import string
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
!gdown 1KU8Jx7On0xmDoIGegxHAnliS8wk2rUq3

Downloading...
From: https://drive.google.com/uc?id=1KU8Jx7On0xmDoIGegxHAnliS8wk2rUq3
To: /content/auteurs.zip
100% 9.75M/9.75M [00:00<00:00, 13.9MB/s]


In [4]:
!unzip "/content/auteurs.zip"

Archive:  /content/auteurs.zip
  inflating: auteurs/.DS_Store       
  inflating: __MACOSX/auteurs/._.DS_Store  
  inflating: auteurs/balzac/M746.xml  
  inflating: __MACOSX/auteurs/balzac/._M746.xml  
  inflating: auteurs/balzac/M754.xml  
  inflating: __MACOSX/auteurs/balzac/._M754.xml  
  inflating: auteurs/balzac/R707.xml  
  inflating: __MACOSX/auteurs/balzac/._R707.xml  
replace auteurs/balzac/M668.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: auteurs/balzac/M668.xml  
  inflating: __MACOSX/auteurs/balzac/._M668.xml  
  inflating: auteurs/balzac/S145.xml  
  inflating: __MACOSX/auteurs/balzac/._S145.xml  
replace auteurs/balzac/M666.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace auteurs/balzac/M666.xml? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: auteurs/balzac/M666.xml  
  inflating: __MACOSX/auteurs/balzac/._M666.xml  
  inflating: auteurs/balzac/S163.xml  
  inflating: __MACOSX/auteurs/balzac/._S163.xml  
  inflating: aut

In [5]:
dumas="auteurs/flaubert/"
balzac="auteurs/balzac/"

In [6]:
def strip_ns_prefix(tree):
    query = "descendant-or-self::*[namespace-uri()!='']"
    for element in tree.xpath(query):
        element.tag = ET.QName(element).localname
    return tree

In [9]:
if balzac != "":
    files = glob.iglob(balzac + '/**/*.xml', recursive=True)
    sentences = []

    for filename in files:
        print(filename)
        parser = ET.XMLParser(remove_blank_text=True, resolve_entities=False, encoding='utf8')
        tree = strip_ns_prefix(ET.parse(filename, parser))

        words = tree.xpath(".//wf/@lemma")

        sentence = []
        for word in words:
            if word != ".":
                sentence.append(word)
            else:
                sentences.append(sentence + [word])
                sentence = []

auteurs/balzac/S139.xml
auteurs/balzac/M758.xml
auteurs/balzac/M885.xml
auteurs/balzac/M749.xml
auteurs/balzac/M746.xml
auteurs/balzac/R707.xml
auteurs/balzac/S163.xml
auteurs/balzac/S145.xml
auteurs/balzac/M763.xml
auteurs/balzac/R681.xml
auteurs/balzac/M754.xml
auteurs/balzac/M666.xml
auteurs/balzac/M668.xml


In [10]:
print(len(sentences))
print(sentences[5])

43925
['le', 'mèche', 'plat', 'de', 'leur', 'long', 'cheveu', 'se', 'unir', 'si', 'habituellement', 'à le', 'poil', 'de', 'le', 'peau', 'de', 'chèvre', 'et', 'cacher', 'si', 'complètement', 'leur', 'visage', 'baisser', 'vert', 'le', 'terre', ',', 'que', 'on', 'pouvoir', 'facilement', 'prendre', 'ce', 'peau', 'pour', 'le', 'leur', ',', 'et', 'confondre', ',', 'à', 'le', 'premier', 'vue', ',', 'ce', 'malheureux', 'avec', 'le', 'animal', 'dont', 'le', 'dépouille', 'lui', 'servir', 'de', 'vêtement', '.']


## Construction du modèle vectoriel

In [12]:
model = Word2Vec(sentences, min_count=2, max_vocab_size=10000, negative=10, epochs=100)

In [14]:
model.wv.save("/content/model_balzac.bin")

In [16]:
print(model.wv.index_to_key)

[',', 'le', 'de', '.', 'un', 'il', 'être', 'et', 'à', 'avoir', 'son', 'ce', 'que', 'en', 'ne', 'je', 'se', '&quot;', 'de le', 'qui', 'vous', '-', 'dans', 'dire', 'pour', '!', 'lui', 'à le', 'pas', 'par', 'mon', 'tout', 'faire', ';', 'me', '?', 'sur', 'si', 'plus', 'comme', 'mais', 'nous', 'avec', 'pouvoir', 'aller', 'où', 'femme', 'homme', 'y', 'voir', 'sans', 'bien', ':', 'vouloir', 'ils', 'moi', 'deux', '...', 'tu', 'leur', 'votre', 'on', 'Mme', 'savoir', 'donner', 'quelque', 'venir', 'devoir', 'dont', 'trouver', 'ou', 'quand', 'Lucien', 'M.', 'mettre', 'petit', 'prendre', 'chez', 'donc', 'grand', 'vie', 'celui', 'fille', 'franc', 'père', 'aimer', 'répondre', 'oeil', 'moment', 'jour', 'te', 'jeune', 'lequel', 'ton', 'après', 'ni', 'beau', 'encore', 'coeur', 'laisser', 'croire', 'autre', 'rien', 'enfant', 'bon', 'demander', 'car', 'amour', 'main', 'vieux', 'aussi', 'reprendre', 'premier', 'regarder', 'jamais', 'alors', 'seul', 'sous', 'toujours', 'vivre', 'monde', 'mère', 'pendant', '

In [17]:
#Paris is to France what London is to what ? model.wv.most_similar(positive=['Londres', 'France'], negative=['Paris'],topn=5)
#King is to man what Queen is to what ? model.wv.most_similar(positive=['reine', 'homme'], negative=['roi'],topn=5)
model.wv.most_similar(positive=['reine', 'homme'], negative=['roi'],topn=10)

[('fille', 0.5213853120803833),
 ('poète', 0.479248970746994),
 ('femme', 0.47510671615600586),
 ('actrice', 0.4740561246871948),
 ('mariée', 0.4543911814689636),
 ('créature', 0.4540271759033203),
 ('marin', 0.4492737054824829),
 ('personne', 0.4289812743663788),
 ('Parisienne', 0.4275044798851013),
 ('dame', 0.4180542826652527)]

In [30]:
model.wv.most_similar('esprit',topn=10)

[('talent', 0.5654381513595581),
 ('idée', 0.5290871858596802),
 ('sentiment', 0.5243231058120728),
 ('vice', 0.5151078104972839),
 ('énergie', 0.5076226592063904),
 ('génie', 0.5053814053535461),
 ('coquetterie', 0.5028355121612549),
 ('verrue', 0.49921488761901855),
 ('séduction', 0.4923187792301178),
 ('intelligence', 0.4871539771556854)]

## Visualization

In [23]:
!gdown 15XszQYD6Ld_eKLPidzETcTQX9-H053xV

Downloading...
From: https://drive.google.com/uc?id=15XszQYD6Ld_eKLPidzETcTQX9-H053xV
To: /content/stopwords.txt
100% 4.59k/4.59k [00:00<00:00, 16.4MB/s]


In [24]:
stops = open("/content/stopwords.txt", encoding="utf-8").read().split("\n")

In [18]:
from sklearn.manifold import TSNE
import plotly.express as px

In [29]:
nb_mots_a_visualiser = 500
words = [word for word in model.wv.index_to_key if word not in stops and word not in string.punctuation][:500]
word_vectors = np.array([model.wv[word] for word in words])

tsne = TSNE(n_components=3, perplexity=30, learning_rate=200,random_state=0)
vectors_3d = tsne.fit_transform(word_vectors)

import pandas as pd
df = pd.DataFrame(vectors_3d, columns=['x', 'y', 'z'])
df['word'] = words

fig = px.scatter_3d(df, x='x', y='y', z='z', text='word', title="Visualisation 3D des Vecteurs Word2Vec")
fig.show()

In [27]:
fig.write_html("/content/word2vec_visualization.html")